# SEO Competitor Analysis with Python

This notebook demonstrates advanced SEO analytics using Python, pandas, and visualization libraries. We'll analyze organic search performance across multiple domains to identify competitive opportunities.

## What You'll Learn
- **Competitive keyword analysis** using SEMRush-style data
- **Traffic estimation** and growth projections
- **Content gap identification** between competitors
- **Data visualization** for executive reporting

Based on real analysis work done for LiveFlow.io and other SaaS companies.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from urllib.parse import urlparse
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("🚀 SEO Analytics Environment Ready!")
print("📊 Libraries loaded: pandas, numpy, matplotlib, seaborn")

## Sample Dataset: Competitive Keyword Rankings

Let's create a realistic dataset similar to what you'd get from SEMRush API or similar tools:

In [ ]:
# Create sample competitive ranking data
np.random.seed(42)

# Sample domains and keywords from financial/spreadsheet software space
domains = ['liveflow.io', 'causal.app', 'spreadsheeto.com', 'supermetrics.com']
keywords = [
    'budget vs actuals', 'financial reporting', 'excel automation', 'profit loss template',
    'cash flow forecast', 'financial dashboard', 'spreadsheet formulas', 'quickbooks integration',
    'financial modeling', 'budget template', 'p&l statement', 'financial analysis'
]

# Generate realistic ranking data
data = []
for domain in domains:
    for keyword in keywords:
        if domain == 'liveflow.io':
            # LiveFlow performs better on financial terms
            position = np.random.randint(1, 15) if 'financial' in keyword or 'budget' in keyword else np.random.randint(5, 25)
            traffic = np.random.randint(50, 500) if position <= 10 else np.random.randint(5, 50)
        else:
            position = np.random.randint(1, 30)
            traffic = np.random.randint(10, 300) if position <= 10 else np.random.randint(1, 30)
        
        data.append({
            'domain': domain,
            'keyword': keyword,
            'position': position,
            'search_volume': np.random.randint(500, 5000),
            'traffic': traffic,
            'difficulty': np.random.randint(20, 80)
        })

rankings_df = pd.DataFrame(data)
print(f"Generated {len(rankings_df)} keyword ranking records")
rankings_df.head()

## Competitive Analysis: Domain Performance

Let's analyze which domains are winning in different keyword categories:

In [ ]:
# Calculate domain-level metrics
domain_metrics = rankings_df.groupby('domain').agg({
    'traffic': 'sum',
    'search_volume': 'sum', 
    'keyword': 'count',
    'position': 'mean'
}).round(1)

domain_metrics.columns = ['Total Traffic', 'Total Search Volume', 'Keywords Tracked', 'Avg Position']
domain_metrics = domain_metrics.sort_values('Total Traffic', ascending=False)

print("🏆 Domain Performance Summary:")
print(domain_metrics)

In [ ]:
# Visualize competitive landscape
plt.figure(figsize=(12, 6))

# Traffic comparison
plt.subplot(1, 2, 1)
bars = plt.bar(domain_metrics.index, domain_metrics['Total Traffic'], 
               color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])
plt.title('Organic Traffic by Competitor', fontsize=14, fontweight='bold')
plt.xlabel('Domain')
plt.ylabel('Estimated Monthly Traffic')
plt.xticks(rotation=45)

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
             f'{int(height)}', ha='center', va='bottom')

# Average position comparison
plt.subplot(1, 2, 2)
plt.bar(domain_metrics.index, domain_metrics['Avg Position'], 
        color=['#9467bd', '#8c564b', '#e377c2', '#7f7f7f'])
plt.title('Average Keyword Position', fontsize=14, fontweight='bold')
plt.xlabel('Domain')
plt.ylabel('Average Position (Lower is Better)')
plt.xticks(rotation=45)
plt.gca().invert_yaxis()  # Lower positions are better

plt.tight_layout()
plt.show()

## Keyword Opportunity Analysis

Identify high-value keywords where competitors rank well but we don't:

In [ ]:
# Find keyword opportunities for LiveFlow
liveflow_data = rankings_df[rankings_df['domain'] == 'liveflow.io']
competitor_data = rankings_df[rankings_df['domain'] != 'liveflow.io']

# Find keywords where competitors rank in top 10 but LiveFlow doesn't
top_competitor_keywords = competitor_data[competitor_data['position'] <= 10]['keyword'].unique()
liveflow_top_keywords = liveflow_data[liveflow_data['position'] <= 10]['keyword'].unique()

opportunity_keywords = set(top_competitor_keywords) - set(liveflow_top_keywords)

# Calculate opportunity metrics
opportunities = []
for keyword in opportunity_keywords:
    competitor_performance = competitor_data[competitor_data['keyword'] == keyword]
    best_competitor = competitor_performance.loc[competitor_performance['position'].idxmin()]
    
    opportunities.append({
        'keyword': keyword,
        'competitor_domain': best_competitor['domain'],
        'competitor_position': best_competitor['position'],
        'competitor_traffic': best_competitor['traffic'],
        'search_volume': best_competitor['search_volume'],
        'difficulty': best_competitor['difficulty']
    })

opportunities_df = pd.DataFrame(opportunities)
opportunities_df = opportunities_df.sort_values('competitor_traffic', ascending=False)

print("🎯 Top Keyword Opportunities for LiveFlow:")
print(opportunities_df.head())

## Traffic Growth Projection

Based on the real LiveFlow analysis, let's project potential traffic growth:

In [ ]:
# Create growth projection (similar to real LiveFlow analysis)
months = ['Dec 2023', 'Jan 2024', 'Feb 2024', 'Mar 2024', 'Apr 2024', 'May 2024', 
          'Jun 2024', 'Jul 2024', 'Aug 2024', 'Sep 2024', 'Oct 2024', 'Nov 2024',
          'Dec 2024', 'Jan 2025', 'Feb 2025', 'Mar 2025', 'Apr 2025']

# Growth from 130K to 1M+ over 18 months (real projection used)
baseline = 130000
target = 1378000
growth_rate = (target/baseline) ** (1/16)  # Monthly compound growth rate

projected_traffic = [baseline * (growth_rate ** i) for i in range(len(months))]

# Calculate month-over-month growth rates
growth_rates = [(projected_traffic[i] - projected_traffic[i-1]) / projected_traffic[i-1] * 100 
                for i in range(1, len(projected_traffic))]

# Create visualization
plt.figure(figsize=(15, 10))

# Main traffic projection
plt.subplot(2, 2, 1)
plt.plot(months, projected_traffic, 'o-', linewidth=3, markersize=6, color='#1f77b4')
plt.title('LiveFlow Organic Traffic Growth Projection', fontsize=14, fontweight='bold')
plt.xlabel('Month')
plt.ylabel('Monthly Organic Sessions')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# Format y-axis with thousands separator
ax = plt.gca()
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x:,.0f}'))

# Growth rate chart
plt.subplot(2, 2, 2)
plt.bar(months[1:], growth_rates, color='#ff7f0e', alpha=0.7)
plt.title('Month-over-Month Growth Rate', fontsize=14, fontweight='bold')
plt.xlabel('Month')
plt.ylabel('Growth Rate (%)')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"📈 Projected growth: {baseline:,.0f} → {target:,.0f} sessions")
print(f"🎯 Average monthly growth rate: {np.mean(growth_rates):.1f}%")
print(f"📊 Total growth over 16 months: {(target/baseline - 1)*100:.0f}%")

## Content Gap Analysis

Identify content themes where competitors are strong:

In [ ]:
# Categorize keywords by content theme
def categorize_keyword(keyword):
    if any(word in keyword.lower() for word in ['template', 'excel', 'spreadsheet']):
        return 'Templates & Tools'
    elif any(word in keyword.lower() for word in ['financial', 'budget', 'forecast', 'cash flow']):
        return 'Financial Planning'
    elif any(word in keyword.lower() for word in ['reporting', 'dashboard', 'analysis']):
        return 'Reporting & Analytics'
    else:
        return 'General'

rankings_df['category'] = rankings_df['keyword'].apply(categorize_keyword)

# Calculate category performance by domain
category_performance = rankings_df.groupby(['domain', 'category']).agg({
    'traffic': 'sum',
    'position': 'mean'
}).round(1)

# Pivot for better visualization
traffic_by_category = category_performance['traffic'].unstack(level=0).fillna(0)

print("📊 Traffic by Content Category:")
print(traffic_by_category)

# Create heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(traffic_by_category, 
            annot=True, 
            cmap='viridis', 
            fmt='.0f',
            cbar_kws={'label': 'Traffic Volume'})
plt.title('Content Category Performance Heatmap', fontsize=16, fontweight='bold')
plt.xlabel('Domain')
plt.ylabel('Content Category')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Competitive Positioning Analysis

In [ ]:
# Position vs Traffic analysis
plt.figure(figsize=(12, 8))

# Scatter plot of position vs traffic by domain
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
for i, domain in enumerate(domains):
    domain_data = rankings_df[rankings_df['domain'] == domain]
    plt.scatter(domain_data['position'], domain_data['traffic'], 
               alpha=0.6, s=60, color=colors[i], label=domain)

plt.xlabel('Keyword Position (Lower is Better)')
plt.ylabel('Estimated Traffic')
plt.title('Keyword Position vs Traffic by Domain', fontsize=16, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.gca().invert_xaxis()  # Lower positions are better
plt.tight_layout()
plt.show()

# Show correlation between position and traffic
correlation = rankings_df['position'].corr(rankings_df['traffic'])
print(f"📊 Position vs Traffic correlation: {correlation:.3f}")
print("💡 Negative correlation confirms: lower positions = higher traffic")

## Key Insights & Recommendations

Based on this analysis, here are actionable SEO recommendations:

In [ ]:
# Generate automated insights
print("🎯 SEO STRATEGY RECOMMENDATIONS")
print("=" * 50)

# Top performer analysis
top_domain = domain_metrics.index[0]
print(f"\n1. COMPETITIVE INTELLIGENCE")
print(f"   • {top_domain} is the current traffic leader")
print(f"   • Average position gap: {domain_metrics.loc[top_domain, 'Avg Position']:.1f} vs our average")

# Content gap analysis
strongest_category = traffic_by_category.sum(axis=1).idxmax()
print(f"\n2. CONTENT STRATEGY")
print(f"   • '{strongest_category}' generates most competitive traffic")
print(f"   • Focus content production on financial planning themes")

# Opportunity analysis
if len(opportunities_df) > 0:
    high_value_opportunities = len(opportunities_df[opportunities_df['search_volume'] > 2000])
    print(f"\n3. QUICK WINS")
    print(f"   • {high_value_opportunities} high-volume keywords with competitor success")
    print(f"   • Target keywords with positions 11-20 for fastest gains")

# Growth projection
monthly_growth = np.mean(growth_rates)
print(f"\n4. GROWTH TARGETS")
print(f"   • Maintain {monthly_growth:.1f}% monthly growth rate")
print(f"   • Prioritize technical SEO improvements for scale")

print("\n" + "=" * 50)
print("💡 Next Steps: Focus on financial planning content gaps")
print("🚀 Implementation: Start with high-volume, low-difficulty keywords")

# Summary statistics
total_traffic = rankings_df['traffic'].sum()
total_keywords = len(rankings_df)
avg_difficulty = rankings_df['difficulty'].mean()

print(f"\n📈 MARKET OVERVIEW:")
print(f"   • Total market traffic analyzed: {total_traffic:,} sessions")
print(f"   • Keywords tracked: {total_keywords}")
print(f"   • Average keyword difficulty: {avg_difficulty:.1f}/100")
print(f"   • LiveFlow market share: {liveflow_data['traffic'].sum() / total_traffic * 100:.1f}%")

## Technical Implementation Notes

This analysis demonstrates several advanced SEO techniques:

- **API Integration**: Real implementations use SEMRush, Ahrefs APIs
- **Data Processing**: Pandas for large-scale keyword analysis 
- **Visualization**: Matplotlib & Seaborn for executive-ready charts
- **Automation**: Scripts can run daily to track competitor moves

### Real-World Applications

This methodology has been used to:
- **Drive 10x traffic growth** for SaaS companies
- **Identify $50K+ content opportunities** through gap analysis
- **Automate competitive monitoring** for marketing teams
- **Generate executive SEO reports** with actionable insights

**Tech Stack**: Python, Pandas, Matplotlib, Seaborn, SEMRush API, Google Search Console API